# Imports & Setup

In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [3]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'ir_assg3_eithan' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if "parquet" in b.name:
        paths.append(full_path+b.name)

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [4]:
parquetFile = spark.read.parquet(*paths)

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [5]:
# Count number of wiki pages
parquetFile.count()

6348910

In [6]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [7]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [8]:
from inverted_index_gcp import InvertedIndex

<H1> Helper function to create Indexes <H1/>

In [9]:
# Put your `generate_graph` function here
def generate_graph(pages):
    distinct_rdd =  pages.map(lambda page: (page[0], set(map(lambda id_text: id_text[0], page[1]))))
    edges = distinct_rdd.flatMap(lambda src_page: list(map(lambda dst_page: (src_page[0], dst_page),src_page[1])))
    vertices = edges.flatMap(lambda edge: [(edge[0],),(edge[1],)]).distinct()
    return edges, vertices

In [ ]:
pages_links = parquetFile.select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/indexes/pr/page_rank_dict.csv', compression="gzip")
pr.show()

+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750|  9913.72878216078|
|  10568| 5385.349263642036|
|  32927| 5282.081575765279|
|  30680| 5128.233709604123|
|5843419|  4957.56768626387|
|  68253| 4769.278265355163|
|  31717| 4486.350180548311|
|  11867|  4146.41465091277|
|  14533| 3996.466440885504|
| 645042| 3531.627089803744|
|  17867|3246.0983906041433|
|5042916| 2991.945739166179|
|4689264| 2982.324883041748|
|  14532| 2934.746829203171|
|  25391| 2903.546223513399|
|   5405| 2891.416329154635|
|4764461|2834.3669873326608|
|  15573|2783.8651181588393|
|   9316|  2782.03964641377|
|8569916| 2775.286191840017|
+-------+------------------+
only showing top 20 rows

